In [ ]:
%load_ext autoreload
%autoreload 2
from label import filter_shot_chains, pos_labeling

from data import load_and_process_metadata, load_and_process_soccer_events

In [ ]:
event_df, players_df = load_and_process_soccer_events(
    "/home/soccerdata/FIFA_WorldCup_2022/Event Data"
)
metadata_df = load_and_process_metadata("/home/soccerdata/FIFA_WorldCup_2022/Metadata")

In [ ]:
pos_chains = pos_labeling(event_df, chain_len=6)

In [ ]:
accepted_chains = filter_shot_chains(
    chains=pos_chains,
    chains_range=(0, 10),
    event_df=event_df,
    players_df=players_df,
    metadata_df=metadata_df,
    output_dir="/home/soccerdata/frames",
    show_video=True,
    interval=1000,
)